In [35]:
!pip install fairseq


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: C:\Users\Post Scriptum\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [36]:
!pip install tensorboardX


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: C:\Users\Post Scriptum\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [37]:
!binarize_nsp.sh acc

In [38]:
!git clone https://github.com/pytorch/fairseq

fatal: destination path 'fairseq' already exists and is not an empty directory.


In [39]:
!cd fairseq;git checkout 336942734c85791a90baa373c212d27e7c722662;pip install --editable .

The system cannot find the path specified.


In [40]:
!cd fairseq;git checkout 336942734c85791a90baa373c212d27e7c722662

The system cannot find the path specified.


In [41]:
%cd fairseq

s:\NLP\PiRhDy\musicbert\fairseq


In [42]:
import sys
import os 

sys.path.append("S:/NLP/PiRhDy/musicbert/fairseq")

from fairseq.models.roberta import RobertaModel

KeyboardInterrupt: 

In [3]:
%cd ../

s:\NLP\PiRhDy


In [14]:
import pickle
import numpy as np
import json
from scipy.sparse import csc_matrix
import copy
import random

random.seed(0)




def reconstruct_sparse(target_dict, name):
    """Return a reconstructed instance of `scipy.sparse.csc_matrix`."""
    return csc_matrix((target_dict[name + '_csc_data'],
                       target_dict[name + '_csc_indices'],
                       target_dict[name + '_csc_indptr']),
                      shape=target_dict[name + '_csc_shape']).toarray()

def load_sequence(file):
    with np.load(file) as loaded:
        if 'meta.json' not in loaded:
            raise ValueError("Cannot find 'info.json' in the npz file.")
        meta_dict = json.loads(loaded['meta.json'].decode('utf-8'))
        time_info = meta_dict['time_signature_changes']
        # melody_id=meta_dict['melody_id']
        idx = 0
        tracks = []
        while str(idx) in meta_dict:
            matrix = reconstruct_sparse(
                loaded, 'track_{}'.format(idx))
            program = meta_dict[str(idx)]
            # tracks.append([program,ticks,matrix])
            tracks.append(matrix)
            idx += 1
        melody_info = reconstruct_sparse(loaded, 'melody_info')
        return time_info, tracks, melody_info





def get_bars(time_info, matrix):
    bars = []
    end_time = len(matrix[0])
    for time_signature in time_info[::-1]:
        signature = int(time_signature[0])
        time = int(time_signature[1])
        temp_matrix = matrix[:, time:end_time]
        bar_number = (end_time - time) // signature
        for i in reversed(range(bar_number)):
            start = time + i * signature
            end = start + signature
            bar_matrix = temp_matrix[:, start:end]
            bar_melody = bar_matrix[0:4, :]
            bar_count_info = len(np.nonzero(bar_melody)[0]) // 4
            if bar_count_info / signature >= 0.75:
                bars.insert(0, temp_matrix[:, start:end])
        end_time = time
    return bars


barlist = {}
def split_sequence(name):
    file_list = pickle.load(open('filenames/sequence/{}'.format(name), 'rb'))
    # split sequence into bars
    bar_path = 'dataset/bar/{}/'.format(name)

    for file in file_list:
        meta_dict = {}
        bar_info = {}
        time_info, tracks, melody_info = load_sequence(file)
        time_info_len = len(time_info)
        current_index = 1
        while current_index < time_info_len:
            if (time_info[current_index][1] - time_info[current_index - 1][1]) < time_info[current_index - 1][0]:
                time_info.remove(time_info[current_index - 1])
                time_info_len -= 1
            current_index += 1
        meta_dict['time_info'] = time_info
        song_matrix = copy.deepcopy(melody_info)
        for idx, track in enumerate(tracks):
            song_matrix = np.vstack((song_matrix, track))
        bars = get_bars(time_info, song_matrix)
        #  filter songs with no more than 8 bars
        if len(bars) >= 8:
            barlist[file]=len(bars)
            for idx, bar in enumerate(bars):
                # update_sparse(bar_info, bar, str(idx))
                bar_info[str(idx)] = bar
        else:
            barlist[file] = 0

            # # save bars
            # np.savez_compressed(bar_path + filename, **bar_info)


In [36]:
meta_dict = {}
bar_info = {}
time_info, tracks, melody_info = load_sequence('./sequence/0/TRAADKW128E079503A_aug_add_agg_neg.npz')
time_info_len = len(time_info)
current_index = 1
while current_index < time_info_len:
    if (time_info[current_index][1] - time_info[current_index - 1][1]) < time_info[current_index - 1][0]:
        time_info.remove(time_info[current_index - 1])
        time_info_len -= 1
    current_index += 1
meta_dict['time_info'] = time_info
song_matrix = copy.deepcopy(melody_info)
print(song_matrix.shape)
print(len(tracks))
for idx, track in enumerate(tracks):
    print(track[3])
    song_matrix = np.vstack((song_matrix, track))
print(song_matrix.shape)
bars = get_bars(time_info, song_matrix)

(4, 4095)
11
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 2 2 1]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 2 2 1]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 2 2 1]
[0 0 0 ... 2 2 1]
[0 0 0 ... 0 0 0]
(48, 4095)


In [ ]:
check 2~nth 'track's, see their last row, if theres any non-zero then instrument exists

In [15]:
dirs = ['0','1','2','3','4','5','6','7','8','9']
for name in dirs:
    split_sequence(name)

In [20]:
print(len(barlist))
# print(barlist.keys())
barlist['sequence/0/TRAADKW128E079503A_aug_add_agg_neg.npz']

930


65

In [1]:
%cd ../

s:\NLP\PiRhDy


In [21]:
import numpy as np
seq = np.load('./sequence/0/TRAADKW128E079503A_aug_add_agg_neg.npz')

print(seq['meta.json'])

# for k in seq.keys():
#     print(k)

# bar = np.load('./dataset/bar/0/TRAADKW128E079503A_aug_add_agg_neg.npz')
# for k in bar.keys():
#     print(k)

phrase = np.load('./dataset/phrase/0/TRAADKW128E079503A_aug_add_agg_neg.npz')
for k in phrase.keys():
    print(k)

# print(bar['0'].shape)
print(phrase['0'].shape)

b'{"time_signature_changes": [[32, 0]], "0": 73, "1": 73, "2": 25, "3": 25, "4": 34, "5": 34, "6": 26, "7": 33, "8": 33, "9": 24, "10": 24}'
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
(48, 128)


In [24]:
import json
meta = json.loads(seq['meta.json'].decode('utf-8'))
for k in meta.keys():
    print(k)

time_signature_changes
0
1
2
3
4
5
6
7
8
9
10


In [40]:
import miditoolkit
midi_obj =  miditoolkit.midi.parser.MidiFile('dataset/lakh_dataset/0/TRAADKW128E079503A_ori.mid')
instruments = midi_obj.instruments
for i in range(len(instruments)):
    inst = instruments[i]
    if inst.program==0:
        aug_inst1 = copy.deepcopy(inst)
        aug_inst2 = copy.deepcopy(inst)
        aug_inst1.program=4
        aug_inst2.program=5
        print(aug_inst1.program)
        print(aug_inst2.program)
        print(inst.program)
        aug_inst1.notes = inst.notes[:int(len(inst.notes)/2)]
        aug_inst2.notes = inst.notes[int(len(inst.notes)/2):]
        instruments.append(aug_inst1)
        instruments.append(aug_inst2)

4
5
0


In [10]:
import pickle
import numpy as np
import random
import os
import json

def check_phrase(matrix, threshold_value):
    count_num = len(np.nonzero(matrix)[0]) // 4
    if count_num / matrix.shape[1] >= threshold_value:
        return True
    return False

dirs = ['0','1','2','3','4','5','6','7','8','9']
total = 0
acc_num_list_total = []
include_list_total = []
test_melody_harmony = []

for name in dirs:
    seq_list = pickle.load(open('filenames/sequence/{}'.format(name), 'rb'))
    file_list = pickle.load(open('filenames/phrase/{}'.format(name), 'rb'))
    for file_id, file in enumerate(file_list):
        acc_num_list = []
        include_list = []
        sequence = np.load(seq_list[file_id], allow_pickle=True)
        phrases = np.load(file, allow_pickle=True)
        meta_dict = json.loads(sequence['meta.json'].decode('utf-8'))
        for idx, k in enumerate(meta_dict.keys()):
            if meta_dict[k] == 0:
                track_index_0 = idx
            elif meta_dict[k] == 25:
                track_index_1 = idx
            elif meta_dict[k] == 33:
                track_index_2 = idx
        phrase_id = 0
        if file_id % 100 not in [0,1,2,3,4,5]:
            while str(phrase_id) in phrases:
                acc_num = 0
                phrase = phrases[str(phrase_id)]
                melody = phrase[0:4, :]
                inst_0 = phrase[4*track_index_0+4:4*track_index_0+8,3]
                inst_1 = phrase[4*track_index_1+4:4*track_index_1+8,3]
                inst_2 = phrase[4*track_index_2+4:4*track_index_2+8,3]

                track_num = phrase.shape[0] // 4
                for track_id in range(1, track_num):
                    accompaniment = phrase[0 + track_id * 4:4 * (track_id + 1), :]
                    if check_phrase(accompaniment, 0.5):
                        test_melody_harmony.append([file_id, melody, accompaniment])
                        acc_num += 1
                if acc_num != 0:
                    acc_num_list.append(acc_num)
                    if sum(inst_0) * sum(inst_1) * sum(inst_2) == 0:
                        include_list.append(0)
                    else:
                        include_list.append(1)
                phrase_id += 1
            include_list_total.append(include_list)
            acc_num_list_total.append(acc_num_list)
print(len(acc_num_list_total))
print(len(include_list_total))
# for l in acc_num_list_total:
#     print(len(l))
print(len(test_melody_harmony))

852
852
84652


In [ ]:
for i in include_list_total.

In [5]:
print(include_list_total)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 

In [12]:
print(len(acc_num_list_total)/6)

142.0


In [9]:
%cd ../

s:\NLP\PiRhDy


In [11]:
%cd musicbert

s:\NLP\PiRhDy\musicbert


In [20]:
# %cd musicbert
import numpy as np
from numpy import dot
from numpy.linalg import norm

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

candidate = ['add_neg','add_pos','change_neg','change_pos','change_shuf_neg','orig']
result = np.loadtxt('logits.txt')
cos_sims = [[],[],[],[],[],[]]

for i in range(len(result)):
    result[i] = softmax(result[i])

# print(result.shape)
start = 0
for i in range(int(len(acc_num_list_total)/6)):
    if sum(include_list_total[6*i]) == 0:
        continue
# for i in range(int(len(acc_num_list_total)/5)):
    logits = []
    for j in range(6):
        logits.append(result[start+len(acc_num_list_total[6*i]) * 50 * j:start+len(acc_num_list_total[6*i]) * 50 * (j+1)])
    start = len(acc_num_list_total[6*i]) * 50 * 6
    print(f'#{6*i+1}')
    for j in range(6):
        cos_sim = []
        avg_logit = []
        # print(len(logits[j]))
        # print(len(include_list_total[6*i]))
        # print(include_list_total[0])
        for k in range(len(logits[j])):
            if include_list_total[6*i][k//50] == 0:
                continue
            avg_logit.append(logits[j][k])
            # if j in [0,2]:
            #     cos_sim.append(dot(logits[j][k], logits[j+1][k])/(norm(logits[j][k])*norm(logits[j+1][k])))    
            # elif j in [1,3,4]:
            #     cos_sim.append(dot(logits[j][k], logits[j-1][k])/(norm(logits[j][k])*norm(logits[j-1][k])))    
            # else:
            #     cos_sim.append(dot(logits[j][k], logits[5][k])/(norm(logits[j][k])*norm(logits[5][k])))
            cos_sim.append(dot(logits[j][k], logits[5][k])/(norm(logits[j][k])*norm(logits[5][k])))
            cos_sims[j].append(dot(logits[j][k], logits[5][k])/(norm(logits[j][k])*norm(logits[5][k])))
        # print(sum(avg_logit)/len(avg_logit))
        print('candidate: ',candidate[j])
        print('max: ',max(cos_sim) if len(cos_sim) > 0 else 0)
        print('min: ',min(cos_sim) if len(cos_sim) > 0 else 0)
        print('mean: ',sum(cos_sim)/len(cos_sim) if len(cos_sim) > 0 else 0)
        print('')


#31
candidate:  add_neg
max:  0.9999999968198715
min:  0.0027664066759920756
mean:  0.6624357807564288

candidate:  add_pos
max:  0.9999999970091404
min:  0.008571636406791669
mean:  0.7360902461913045

candidate:  change_neg
max:  0.9999999586936003
min:  0.011822212911816276
mean:  0.7278725530948527

candidate:  change_pos
max:  0.9999999992567764
min:  0.015271239872416421
mean:  0.6429248297580894

candidate:  change_shuf_neg
max:  0.9999999999996353
min:  0.0021139256508415864
mean:  0.7406668211572205

candidate:  orig
max:  1.0000000000000002
min:  0.9999999999999998
mean:  1.0

#43
candidate:  add_neg
max:  0.9999999998215493
min:  0.000748842497495398
mean:  0.7125630554153203

candidate:  add_pos
max:  0.9999999999728407
min:  0.00032859527271932086
mean:  0.6894655371989661

candidate:  change_neg
max:  0.9999999988547401
min:  0.00045195690651828047
mean:  0.7049704296297945

candidate:  change_pos
max:  0.999999999853156
min:  0.000626402408430329
mean:  0.733115771861830

In [13]:
print(len(candidate))

6


In [17]:
print(len(cos_sims))

6


In [21]:
import statistics
for j in range(6):
    print('candidate: ',candidate[j])
    print('max: ',max(cos_sims[j]) if len(cos_sims[j]) > 0 else 0)
    print('min: ',min(cos_sims[j]) if len(cos_sims[j]) > 0 else 0)
    print('mean: ',sum(cos_sims[j])/len(cos_sims[j]) if len(cos_sims[j]) > 0 else 0)
    print('median: ',statistics.median(cos_sims[j]) if len(cos_sims[j]) > 0 else 0)
    print('')

candidate:  add_neg
max:  1.0000000000000002
min:  0.0003586550744526164
mean:  0.7400226143283389
median:  0.9905568397032111

candidate:  add_pos
max:  0.9999999999999762
min:  0.00032859527271932086
mean:  0.7463896413716502
median:  0.9888806984612513

candidate:  change_neg
max:  0.9999999999999972
min:  0.0003455272782113879
mean:  0.7518376676169964
median:  0.9894729360262944

candidate:  change_pos
max:  1.0000000000000002
min:  0.00033895561433913625
mean:  0.7504921664794248
median:  0.993299692965823

candidate:  change_shuf_neg
max:  1.0000000000000002
min:  0.0003750268130960397
mean:  0.7608272631145576
median:  0.9948916788700102

candidate:  orig
max:  1.0000000000000002
min:  0.9999999999999998
mean:  1.0
median:  1.0



: 

0